# Introduction
Source code for exercise 3 - Fine-tuning Language Model

## Set up

In [1]:
!pip install transformers datasets torch matplotlib seaborn

In [2]:
from datasets import load_dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
import torch

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cuda:0


In [4]:
# Load dataset
dataset = load_dataset('ag_news')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

## Description of dataset

In [ ]:
# 1. Check the structure (Splits)
print(dataset)

# 2. Check the class names
features = dataset['train'].features
print(f"Classes: {features['label'].names}")

# 3. Look at a sample structure
print(f"Sample text: {dataset['train'][0]['text']}")
print(f"Sample label: {dataset['train'][0]['label']} ({features['label'].names[dataset['train'][0]['label']]})")

## 3.2 Stats and example

In [ ]:
import pandas as pd

class_names = dataset['train'].features['label'].names

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

def get_stats(df):
    # Calculate samples per class
    class_counts = df.value_counts('label')

    dist = {class_names[i]: int(class_counts[i]) for i in range(len(class_names))}

    # Calculate text length statistics (number of characters)
    df['text_len'] = df['text'].apply(len)
    avg_len = df['text_len'].mean()

    return dist, avg_len

# 2. Generate Statistics
train_dist, train_avg_len = get_stats(train_df)
test_dist, test_avg_len = get_stats(test_df)

print("--- Summary Statistics ---")
print(f"Total Training Samples: {len(dataset['train'])}")
print(f"Total Testing Samples: {len(dataset['test'])}")
print(f"\nTraining Class Distribution: {train_dist}")
print(f"Testing Class Distribution: {test_dist}")
print(f"\nAvg. Text Length (Train): {train_avg_len:.2f} characters")

# 3. Representative Examples
print("\n--- Representative Examples ---")
for i in range(len(class_names)):
  example = train_df[train_df['label'] == i].iloc[0]['text']
  print(f'{class_names[i]}: {example}')



## Fine tuning

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

# Load dataset
dataset = load_dataset('ag_news')

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Preprocess data
def preprocess_function(examples):
    # Only tokenize and truncate, padding will be handled by DataCollatorWithPadding
    tokenized_inputs = tokenizer(examples['text'], truncation=True)
    tokenized_inputs["labels"] = examples["label"] # Add labels back as Trainer expects them
    return tokenized_inputs

# Apply preprocessing
encoded_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names # This removes 'text' and 'label'
)

# Initialize DataCollatorWithPadding
# This collator will pad the sequences to the longest in each batch and convert to PyTorch tensors
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

# Initialize Trainer with the data_collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    data_collator=data_collator # Pass the data collator here
)

# Start training
trainer.train()

trainer.save_model("./fine-tuned-bert")
tokenizer.save_pretrained("./fine-tuned-bert")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

model = BertModel.from_pretrained("bert-base-uncased", output_attentions=True)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Example input
inputs = tokenizer("The movie was fantastic!", return_tensors="pt")

# Get attentions
outputs = model(**inputs)
attentions = outputs.attentions # List of attention maps for each layer